In [1]:
import sys
cpath = !pwd
#sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')

In [2]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder
from sklearn import preprocessing
from datasets.dataset import Dataset
from lionforests import LionForests

In [3]:
abalone = Dataset()
X, y, feature_names, class_names = abalone.load_abalone(type='regression')

In [4]:
X, y, feature_names, class_names

(array([[0, 0.455, 0.365, ..., 0.2245, 0.10099999999999999, 0.15],
        [0, 0.35, 0.265, ..., 0.0995, 0.0485, 0.07],
        [1, 0.53, 0.42, ..., 0.2565, 0.1415, 0.21],
        ...,
        [0, 0.6, 0.475, ..., 0.5255, 0.2875, 0.308],
        [1, 0.625, 0.485, ..., 0.531, 0.261, 0.29600000000000004],
        [0, 0.71, 0.555, ..., 0.9455, 0.3765, 0.495]], dtype=object),
 array([15,  7,  9, ...,  9, 10, 12], dtype=int64),
 ['Sex', 'Length', 'Diam', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell'],
 ['Age (+1.5)'])

In [11]:
LF = LionForests(None, False, None, list(feature_names), class_names=class_names, categorical_features=None)
parameters = [{
    'max_depth': [10],
    'max_features': [0.75],
    'bootstrap': [True],
    'min_samples_leaf' : [5],
    'n_estimators': [100]
}]
LF.fit(X, y, params=parameters)

In [12]:
LF.error

-1.4430574896370096

In [9]:
LF.model, LF.model.bootstrap, LF.model.n_estimators

(RandomForestRegressor(max_depth=10, max_features=0.75, min_samples_leaf=5,
                       n_jobs=-1, random_state=0),
 True,
 100)

In [13]:
LF.explain(X[200], reduction=True, ar_algorithm=None, cl_algorithm=None, save_plots=False, instance_qe=0, clusters=0, method = 'R1')

['if 0.204<=Shell<=0.207 & 0.241<=Shucked<=0.241 & 0.137<=Height<=0.142 & 0.611<=Whole<=0.616 & 0.132<=Viscera<=0.136 & 0.498<=Length<=0.508 then quality: 11.3538 +/- 1.3071 error',
 100,
 8,
 45,
 6,
 [{'Height': [['<=', 0.542553186416626]],
   'Shucked': [['<=', 0.2424616888165474]],
   'Whole': [['<=', 0.33329586684703827]],
   'Length': [['<=', 0.6008403599262238], ['>', 0.5504201948642731]],
   'Viscera': [['<=', 0.291158527135849]],
   'Shell': [['<=', 0.3146153837442398], ['>', 0.2834615260362625]]},
  {'Height': [['<=', 0.6276595592498779], ['>', 0.521276593208313]],
   'Shucked': [['<=', 0.23999010771512985]],
   'Shell': [['<=', 0.42692308127880096], ['>', 0.2834615260362625]],
   'Length': [['<=', 0.5756302773952484]],
   'Viscera': [['<=', 0.31554877758026123], ['>', 0.21544715762138367]],
   'Whole': [['<=', 0.27776528894901276]]},
  {'Shell': [['<=', 0.319230780005455], ['>', 0.2576923072338104]],
   'Shucked': [['<=', 0.23900148272514343]],
   'Whole': [['<=', 0.27079586

In [17]:
from tqdm import tqdm
import time

results = []
iterr = 100
with tqdm(total=5*4*2*4*5) as pbar:
    for err in [0, 0.01, 0.1, 1, 10]:
        for i in [1, 5, 7, 10]:
            for j in [10, 100, 500, 1000]:
                for o in ['sqrt', 'log2', 0.75, None]:
                    parameters = [{
                        'max_depth': [i],
                        'max_features': [o],
                        'bootstrap': [True],
                        'min_samples_leaf': [5],
                        'n_estimators': [j]
                    }]
                    lf = LionForests(None, False, None, list(feature_names), class_names=class_names, categorical_features=None)
                    lf.fit(X, y, params=parameters)
                    lf.error
                    tlf = 0
                    fr = 0
                    pr = 0
                    for inde in range(iterr):
                        t_a = time.time()
                        a, b, c, d, e, f, g = lf.explain(
                            X[inde], False, None, None, False)
                        fr = fr + c - e
                        pr = pr + b - d
                        tlf = tlf + time.time() - t_a
                    results.append([str(i), str(j), str(o), 'NoRed','_', '_', tlf/iterr, fr/iterr, pr/iterr, 0, 0])
                    pbar.update(1)
                    error = err*(y.max()-y.min())/100
                    if error == 0:
                        error = abs(lf.error)
                    for k in ['R1', 'R2', 'R3']:
                        if 'R1' == k:
                            for ara in ['apriori', 'fpgrowth']:
                                tlf = 0
                                fr = 0
                                pr = 0
                                lr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, f, g = lf.explain(
                                        X[inde], True, ara,  instance_qe=error, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    if g is not None:
                                        lr = lr + abs(g)
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i), str(j), str(
                                    o), k, ara, '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                                pbar.update(1)
                        else:
                            tlf = 0
                            fr = 0
                            pr = 0
                            lr = 0
                            for inde in range(iterr):
                                t_a = time.time()
                                a, b, c, d, e, f, g = lf.explain(
                                    X[inde], True, None, None,  instance_qe=error, method=k)
                                fr = fr + c - e
                                pr = pr + b - d
                                if g is not None:
                                    lr = lr + abs(g)
                                tlf = tlf + time.time() - t_a
                            results.append([str(i), str(j), str(
                                o), k, '_', '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                            pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████████| 800/800 [5:35:29<00:00, 25.16s/it]


In [18]:
results

[['1', '10', 'sqrt', 'NoRed', '_', '_', 0.22826855421066283, 0.0, 0.0, 0, 0],
 ['1',
  '10',
  'sqrt',
  'R1',
  'apriori',
  '_',
  0.23481747150421142,
  0.0,
  0.0,
  1.8464863916412586,
  0.0],
 ['1',
  '10',
  'sqrt',
  'R1',
  'fpgrowth',
  '_',
  0.23860974311828614,
  0.0,
  0.0,
  1.8464863916412586,
  0.0],
 ['1',
  '10',
  'sqrt',
  'R2',
  '_',
  '_',
  0.23496733188629151,
  1.58,
  4.04,
  1.8464863916412586,
  1.1648306683950014],
 ['1',
  '10',
  'sqrt',
  'R3',
  '_',
  '_',
  0.2324684166908264,
  0.33,
  3.92,
  1.8464863916412586,
  1.276127227802134],
 ['1', '10', 'log2', 'NoRed', '_', '_', 0.22970005989074707, 0.0, 0.0, 0, 0],
 ['1',
  '10',
  'log2',
  'R1',
  'apriori',
  '_',
  0.23842729091644288,
  0.0,
  0.0,
  1.837493678275423,
  0.0],
 ['1',
  '10',
  'log2',
  'R1',
  'fpgrowth',
  '_',
  0.2376285481452942,
  0.0,
  0.0,
  1.837493678275423,
  0.0],
 ['1',
  '10',
  'log2',
  'R2',
  '_',
  '_',
  0.23109861373901366,
  1.59,
  4.72,
  1.837493678275423